In [1]:
import os
import pandas as pd
from func import download
from params import dresden_images_root, dresden_csv, train_csv_path, unseen_csv_path

In [2]:
# Create output folder if needed
if not os.path.exists(dresden_images_root):
    os.makedirs(dresden_images_root)

data = pd.read_csv(dresden_csv)

train_model = ['Ixus70', 'D200', 'mju-1050SW']
unseen_model = ['Ixus55', 'D70', 'PowerShotA640', 'DC-830i', 'DSC-W170']

train_data = data[([model in train_model for model in data['model']])]
unseen_data = data[([model in unseen_model for model in data['model']])]

In [3]:
download(train_data, dresden_images_root, train_csv_path)

100%|██████████| 2359/2359 [09:14<00:00,  4.25it/s]

Number of images: 2359
Saving db to csv


In [3]:
download(unseen_data, dresden_images_root, unseen_csv_path)

  0%|          | 0/1549 [00:00<?, ?it/s]

  0%|          | 1/1549 [00:00<15:11,  1.70it/s]

  0%|          | 2/1549 [00:01<18:51,  1.37it/s]

  1%|          | 12/1549 [00:03<03:23,  7.56it/s]

  1%|          | 13/1549 [00:03<06:33,  3.91it/s]

...

 96%|█████████▌| 1484/1549 [08:56<00:45,  1.44it/s]

 96%|█████████▌| 1485/1549 [08:57<00:48,  1.32it/s]

 96%|█████████▌| 1486/1549 [08:57<00:49,  1.27it/s]

100%|██████████| 1549/1549 [09:12<00:00,  2.80it/s]

Number of images: 1549
Saving db to csv
